In [2]:
dataset_name = 'monuseg'
device = 'cuda'
split_num = 1

import torch
import os
import numpy as np
from Netlib.HoVerNetlib.config import hovernet_config
from Netlib.HoVerNetlib.infer import HovernetInfer
from Netlib.CDNetlib.tools.augmentation import MyCenterCrop
from Netlib.HoVerNetlib.tools.dataset import HoverNetDataset_MoNuSeg

arg = hovernet_config.HoverNetMoNuSeg(ds=dataset_name,device=device,ds_num=split_num)
arg.img_size = [1000,1000]
infer =HovernetInfer(arg)
data = HoverNetDataset_MoNuSeg(arg.DATASET_FP,[1000,1000],tranfs=[MyCenterCrop((1000,1000))],train=False)

preds = np.zeros((0,1000,1000,2),dtype=np.int32)
for i in range(len(data)):
    img = data[i][0].unsqueeze(0).to(device)
    pred = infer(img)
    print(pred.shape)
    preds = np.concatenate((preds,pred),axis=0)

pred_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/gts.npy'
gts_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/preds.npy'
if not os.path.exists(os.path.dirname(pred_path)):
        os.makedirs(os.path.dirname(pred_path))
if not os.path.exists(os.path.dirname(gts_path)):
        os.makedirs(os.path.dirname(gts_path))


from Netlib.HoVerNetlib.tools.utils import draw_instance_map
from Netlib.HoVerNetlib.tools.metric import dice_score

def print_metric(pred_path, true_path):
        pred =np.load(pred_path)
        target = np.load(true_path)
        pred = torch.from_numpy(pred.astype(np.int16)).long() # [N, 256, 256, 2]
        target = torch.from_numpy(target.astype(np.int16)).long() # [N, 256, 256, 2]
        dice = dice_score(pred[..., 1], target[..., 1])
        print(f"dice score:{dice:.6f}")

np.save(pred_path,preds)
cp = MyCenterCrop((1000,1000))

gts = cp((data.labels))[0].permute(0,2,3,1).cpu().numpy()
np.save(gts_path,gts)
imgs = cp((data.imgs))[0]

if not os.path.exists(arg.figure_dir+'/preds'):
        print("not find the path")
        os.makedirs(arg.figure_dir+'/preds')

draw_instance_map(imgs,preds,fp=arg.figure_dir+'/preds')
print_metric(pred_path,gts_path)

print(pred_path)
print(gts_path)
print(arg.CLASSES-1)
os.system(f'python compute_metric/compute_stats.py --mode=seg_class --pred={pred_path} --true={gts_path} --class_num={arg.CLASSES-1}')


Loading dataset from data/monuseg
Dataset loaded
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
(1, 1000, 1000, 2)
dice score:0.822466
numpy_result/monuseg/1/HoverNetMoNuSeg_test_result/gts.npy
numpy_result/monuseg/1/HoverNetMoNuSeg_test_result/preds.npy
1


 93%|█████████▎| 13/14 [01:53<00:07,  7.35s/it]

      pq       SQ       DQ      AJI  multi_pq+
0.632608 0.767365 0.823303 0.655751   0.634868


100%|██████████| 14/14 [02:00<00:00,  8.64s/it]


0

In [3]:
dataset_name = 'cpm'
device = 'cuda'
split_num = 1

import torch
import os
import numpy as np
from Netlib.HoVerNetlib.config import hovernet_config
from Netlib.HoVerNetlib.infer import HovernetInfer
from Netlib.CDNetlib.tools.augmentation import MyCenterCrop
from Netlib.HoVerNetlib.tools.dataset import HoverNetDataset_CPM

arg = hovernet_config.HoverNetCPM(ds=dataset_name,device=device,ds_num=split_num)
infer =HovernetInfer(arg)
data = HoverNetDataset_CPM(arg.DATASET_FP,tranfs=[MyCenterCrop((256,256))],train=False)

preds = np.zeros((0,256,256,2),dtype=np.int32)
for i in range(len(data)):
    img = data[i][0].unsqueeze(0).to(device)
    pred = infer(img)
    preds = np.concatenate((preds,pred),axis=0)

pred_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/gts.npy'
gts_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/preds.npy'
if not os.path.exists(os.path.dirname(pred_path)):
        os.makedirs(os.path.dirname(pred_path))
if not os.path.exists(os.path.dirname(gts_path)):
        os.makedirs(os.path.dirname(gts_path))


from Netlib.HoVerNetlib.tools.utils import draw_instance_map
from Netlib.HoVerNetlib.tools.metric import dice_score

def print_metric(pred_path, true_path):
        pred =np.load(pred_path)
        target = np.load(true_path)
        pred = torch.from_numpy(pred.astype(np.int16)).long() # [N, 256, 256, 2]
        target = torch.from_numpy(target.astype(np.int16)).long() # [N, 256, 256, 2]
        dice = dice_score(pred[..., 1], target[..., 1])
        print(f"dice score:{dice:.6f}")

np.save(pred_path,preds)

gts = np.load('data/cpm17/test/labels.npy')
np.save(gts_path,gts)
imgs = torch.load('data/cpm17/test/imgs.pt')

if not os.path.exists(arg.figure_dir+'/preds'):
        print("not find the path")
        os.makedirs(arg.figure_dir+'/preds')

draw_instance_map(imgs,preds,fp=arg.figure_dir+'/preds')
print_metric(pred_path,gts_path)

print(pred_path)
print(gts_path)
print(arg.CLASSES-1)
os.system(f'python compute_metric/compute_stats.py --mode=seg_class --pred={pred_path} --true={gts_path} --class_num={arg.CLASSES-1}')


Loading dataset from data/cpm17
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
(1, 256, 256, 2)
dice score:0.874094
numpy_result/cpm/1/HoverNetCPM_test_result/gts.npy
numpy_result/cpm/1/HoverNetCPM_test_result/preds.npy
1


 91%|█████████ | 29/32 [00:01<00:00, 26.36it/s]

      pq      SQ       DQ      AJI  multi_pq+
0.664632 0.80368 0.825135 0.711334   0.667397


100%|██████████| 32/32 [00:01<00:00, 25.78it/s]


0

In [1]:
dataset_name = 'dsb'
device = 'cuda'
split_num = 1

import torch
import os
import numpy as np
from Netlib.HoVerNetlib.config import hovernet_config
from Netlib.HoVerNetlib.infer import HovernetInfer
from Netlib.CDNetlib.tools.augmentation import MyCenterCrop
from Netlib.HoVerNetlib.tools.dataset import HoverNetDataset_DSB,Subset

arg = hovernet_config.HoverNetDSB(ds=dataset_name,device=device,ds_num=split_num)
infer =HovernetInfer(arg)
data = HoverNetDataset_DSB(arg.DATASET_FP,tranfs=[MyCenterCrop((256,256))],train=True)

preds = np.zeros((0,256,256,2),dtype=np.int32)
for i in arg.TEST_SET_INDEX:
    img = data[i][0].unsqueeze(0).to(device)
    pred = infer(img)
    preds = np.concatenate((preds,pred),axis=0)

pred_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/gts.npy'
gts_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/preds.npy'
if not os.path.exists(os.path.dirname(pred_path)):
        os.makedirs(os.path.dirname(pred_path))
if not os.path.exists(os.path.dirname(gts_path)):
        os.makedirs(os.path.dirname(gts_path))

import PIL.Image as Image
from torchvision.transforms.functional import pil_to_tensor
from Netlib.HoVerNetlib.tools.utils import draw_instance_map
from Netlib.HoVerNetlib.tools.metric import dice_score

def print_metric(pred_path, true_path):
        pred =np.load(pred_path)
        target = np.load(true_path)
        pred = torch.from_numpy(pred.astype(np.int16)).long() # [N, 256, 256, 2]
        target = torch.from_numpy(target.astype(np.int16)).long() # [N, 256, 256, 2]
        dice = dice_score(pred[..., 1], target[..., 1])
        print(f"dice score:{dice:.6f}")
cp = MyCenterCrop((256,256))
np.save(pred_path,preds)
print(arg.TEST_SET_INDEX)
imgs = torch.zeros((0,3,256,256),dtype=torch.float32)
gts = np.zeros((0,256,256,2),dtype=np.int32)

for i in arg.TEST_SET_INDEX:
    img = Image.open(os.path.join('data/DSB2018/train', data.data_dir[i], 'images', data.data_dir[i]+'.png')).convert('RGB')
    img = pil_to_tensor(img).float() / 255
    img = cp((img))[0]
    instances = os.listdir(os.path.join( 'data/DSB2018/train',data.data_dir[i], "masks"))
    instances.sort()
    label = torch.zeros(2, img.shape[1], img.shape[2], dtype=torch.long)
    for idx, ins in enumerate(instances):
        mask = Image.open(os.path.join(data.train_dir, data.data_dir[i], "masks", ins))
        mask = pil_to_tensor(mask).bool() * 1
        mask = cp((mask))[0]
        label[0][label[0]==0] += mask[0][label[0]==0] * (idx+1)
        label[1][label[1]==0] += mask[0][label[1]==0].long()

    imgs = torch.cat((imgs,img.unsqueeze(0)),0)
    gts = np.concatenate((gts,label.permute(1,2,0).unsqueeze(0).cpu().numpy()),axis=0)

np.save(gts_path,gts)

if not os.path.exists(arg.figure_dir+'/preds'):
        print("not find the path")
        os.makedirs(arg.figure_dir+'/preds')

draw_instance_map(imgs,preds,fp=arg.figure_dir+'/preds')
print_metric(pred_path,gts_path)

print(pred_path)
print(gts_path)
print(arg.CLASSES-1)
os.system(f'python compute_metric/compute_stats.py --mode=seg_class --pred={pred_path} --true={gts_path} --class_num={arg.CLASSES-1}')


Loading dataset from data/DSB2018


/root/miniconda3/lib/python3.8/site-packages/scipy/ndimage/_measurements.py:1534: RuntimeWarning: invalid value encountered in double_scalars
  results = [sum(input * grids[dir].astype(float), labels, index) / normalizer


tensor([659, 439, 334, 558, 271, 414, 195, 652,  23, 409, 292, 111, 287, 324,
        592, 390, 612, 648, 340, 615, 660, 108,   2, 223, 134, 495,  46,  15,
        275, 498, 216, 132, 500, 163, 123, 452,  51, 341, 376, 370, 655,  24,
        211,  80, 526, 496, 446, 385, 173, 548, 471, 561, 209, 305,  37, 551,
        247, 588, 176, 604, 233, 189,  68, 333, 515,  42, 208, 139, 194,  69,
        432, 627, 589, 295, 595, 465, 646,  34,  38, 383, 162,  57, 538,  64,
        477, 613,  85, 605,   8, 213, 510, 293, 265, 261, 210, 481, 356, 408,
        323, 583,  75, 528, 336, 621,  43, 235, 529, 568, 180, 114,  36, 364,
        405, 620, 144, 351, 168, 472,  20, 120, 518, 177, 397, 142, 191, 278,
        649, 299, 447, 463, 601, 519, 527,  56])
dice score:0.896611
numpy_result/dsb/1/HoverNetDSB_test_result/gts.npy
numpy_result/dsb/1/HoverNetDSB_test_result/preds.npy
1


 98%|█████████▊| 131/134 [00:11<00:00, 16.92it/s]

      pq       SQ       DQ      AJI  multi_pq+
0.710686 0.846276 0.824751 0.743456   0.699107


100%|██████████| 134/134 [00:11<00:00, 11.42it/s]


0